In [ ]:
"""
01, cap at .99, 0.387178, 0.384541
"""

In [ ]:
"""
categorical only + various estimator
numerical only + various estimator
variable selection (xgb, RF) + various estimator
numerical as categorical
second order features
standardize kfold transformer
numerical as categorical + 2nd order + linear
"""

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xgboost as xgb
import vw_utils as vw
import gini
%matplotlib inline

In [4]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

cap = train.Hazard.quantile(0.99)
print(cap)

train['Hazard_cap'] = np.minimum(train.Hazard, cap)

data = pd.concat([train, test], axis=0).reset_index()
train_set = np.hstack([np.ones(train.shape[0]), np.zeros(test.shape[0])])

19.0


## --------------------- Feature Start -----------------------

In [5]:
# start
categorical = ['T1_V' + str(i) for i in list(range(4, 10)) + [11, 12, 15, 16, 17]] + \
                ['T2_V' + str(i) for i in [3, 5, 11, 12, 13]]

numerical = set(train.columns).difference(categorical + ['Hazard', 'Hazard_cap', 'Id'])
numerical = list(numerical)

In [6]:
# raw feature
from sklearn.preprocessing import StandardScaler
import re

SS = StandardScaler()
feat_raw = data[numerical].applymap(float)
feat_raw[numerical] = SS.fit_transform(feat_raw[numerical])


toDrop = re.sub('[ ]+', '_', data[categorical].ix[0, :].to_string()).split('\n')
feat_raw_cat = pd.get_dummies(data[categorical]).drop(toDrop, axis=1)

feat_raw = pd.concat([feat_raw, feat_raw_cat], axis=1)
feat_raw.shape

(101999, 95)

In [101]:
# count features
a = data.copy()
tmp = data.T1_V11.astype('category')
tmp = tmp.cat.rename_categories(np.random.randint(0, 10000, 12))
tmp.values

[3513, 3513, 7949, 7949, 7949, ..., 3513, 7949, 3357, 3513, 3513]
Length: 101999
Categories (12, int64): [738, 3513, 4211, 3357, ..., 3363, 7908, 6272, 9941]

In [7]:
# assemble features

tmp = pd.concat([feat_raw], axis=1)
train_c = tmp[train_set==1]
test_c = tmp[train_set==0]

print(train_c.shape)
print(test_c.shape)

(50999, 95)
(51000, 95)


## --------------------- Modeling -----------------------

In [12]:
%%time

import gini
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold

param = {'max_depth':7, 'eta':0.01, 'objective':'reg:linear',
         'gamma':0, 'min_child_weight': 30, 'subsample': 0.8,
         'colsample_bytree': 0.5}

scores = []
time = 1
for idx_train, idx_test in KFold(train.shape[0], 10):
    print(time)
    time += 1
    dTrain = xgb.DMatrix(train_c.ix[idx_train], label=train.Hazard_cap[idx_train])
    dTest = xgb.DMatrix(train_c.ix[idx_test])
    model = xgb.train(param, dTrain, 1000)
    pred = model.predict(dTest)
    scores.append(gini.normalized_gini(train.Hazard[idx_test], pred))

print(np.mean(scores))

1
2
3
4
5
6
7
8
9
10
0.385757196261
CPU times: user 9min 17s, sys: 1.61 s, total: 9min 19s
Wall time: 2min 26s


In [13]:
%%time

import gini
import xgboost as xgb

param = {'max_depth':7, 'eta':0.01, 'objective':'reg:linear',
         'gamma':0, 'min_child_weight': 30, 'subsample': 0.8,
         'colsample_bytree': 0.5}

dTrain = xgb.DMatrix(train_c, label=train.Hazard_cap)
dTest = xgb.DMatrix(test_c)
model = xgb.train(param, dTrain, 1000)
yhat = model.predict(dTest)

(pd.DataFrame({'Id': test.Id, 'Hazard': yhat})
 .reindex_axis(['Id', 'Hazard'], 1)
 .to_csv('../output/xgb_v2_01.csv', index=0))

CPU times: user 59.4 s, sys: 180 ms, total: 59.6 s
Wall time: 16.1 s


In [44]:
pd.DataFrame({'Id': test.Id, 'Hazard': yhat}).reindex_axis(['Id', 'Hazard'], 1).to_csv('../output/ensemble_07.csv', index=0)